<a href="https://colab.research.google.com/github/cbalkig/Anomaly_Detection_in_Videos/blob/master/eval_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install keras_layer_normalization

  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.14.0-cp36-none-any.whl size=5267 sha256=ddf5cc8fa39af80690528213e5bbfe0f5db15cbd7a98aebaeaf9850651983ff6
  Stored in directory: /root/.cache/pip/wheels/54/80/22/a638a7d406fd155e507aa33d703e3fa2612b9eb7bb4f4fe667
Successfully built keras-layer-normalization


In [11]:
import os
from os import listdir
from os.path import join, isdir
import numpy as np
from PIL import Image
import keras
from keras.layers import Conv2DTranspose, ConvLSTM2D, TimeDistributed, Conv2D
from keras.models import Sequential, load_model
from keras_layer_normalization import LayerNormalization
import matplotlib.pyplot as plt

working_directory = '/content/drive/MyDrive/AnomalyDetectionInVideos'


class Config:
    SINGLE_TEST_PATH = os.path.join(working_directory, "files", "769a3c79-f730-4f05-becf-ee9535131173")
    BATCH_SIZE = 4
    MODEL_PATH = os.path.join(working_directory, "model.hdf5")
    FRAME_SIZE = 10
    THRESHOLD = 150


def get_model():
    return load_model(Config.MODEL_PATH, custom_objects={'LayerNormalization': LayerNormalization})


def get_single_test():
    test = np.zeros(shape=(Config.THRESHOLD, 256, 256, 1))
    cnt = 0
    for f in sorted(listdir(Config.SINGLE_TEST_PATH)):
        directory_path = join(Config.SINGLE_TEST_PATH, f)
        if isdir(directory_path):
            for c in listdir(directory_path):
              if len(c) < 7:
                os.rename(join(directory_path, c), join(directory_path, c.zfill(7)))

    for f in sorted(listdir(Config.SINGLE_TEST_PATH)):
        if str(join(Config.SINGLE_TEST_PATH, f))[-3:] == "tif":
            img = Image.open(join(Config.SINGLE_TEST_PATH, f)).resize((256, 256))
            img = np.array(img, dtype=np.float32) / 256.0
            test[cnt, :, :, 0] = img
            cnt = cnt + 1
            if cnt >= Config.THRESHOLD:
                break
    return test


def evaluate(model):
    print("got model")
    test = get_single_test()
    print("got test")
    sz = test.shape[0] - Config.FRAME_SIZE
    sequences = np.zeros((sz, Config.FRAME_SIZE, 256, 256, 1))
    # apply the sliding window technique to get the sequences
    for i in range(0, sz):
        clip = np.zeros((Config.FRAME_SIZE, 256, 256, 1))
        for j in range(0, Config.FRAME_SIZE):
            clip[j] = test[i + j, :, :, :]
        sequences[i] = clip

    # get the reconstruction cost of all the sequences
    reconstructed_sequences = model.predict(sequences, batch_size=Config.BATCH_SIZE)
    sequences_reconstruction_cost = np.array([np.linalg.norm(np.subtract(sequences[i],reconstructed_sequences[i])) for i in range(0, sz)])
    sr = sequences_reconstruction_cost

    print(sr)


if __name__ == '__main__':
    model = get_model()
    evaluate(model)

got model
got test
[60.58769852 60.53352542 60.74483954 60.69037406 60.74722808 60.88212494
 60.84381953 60.52337918 60.32696724 60.2001196  60.09024327 59.75315001
 59.93210079 59.83973635 60.02817638 60.39261675 60.41411568 60.21996958
 60.73135519 61.03648709 61.31917324 61.79496497 61.74073227 61.99356712
 61.53382237 61.91758764 61.54932902 62.32790512 62.4216156  63.03949565
 63.5259464  63.82729697 65.14135137 65.55129968 66.8755509  67.24378201
 68.21947433 68.13962948 68.80590695 69.09964903 69.8661556  70.28850867
 70.51527807 70.65336848 70.80406728 70.87071689 70.9278248  71.00175519
 71.23838518 71.02581699 70.68785789 71.06879737 70.96860242 70.90145973
 70.82905514 70.94676529 70.67287355 70.19591835 69.56188328 69.36154529
 69.18364472 68.89052785 68.89040434 68.58944157 68.87541284 69.41042921
 69.80923524 70.27810838 69.63450048 69.5364777  69.25122023 68.86508588
 67.95428478 67.41475576 66.93749973 66.1887199  65.35544272 64.48743331
 64.40470668 64.18873966 64.3068